# Load Environment
The `.env` file is expected to include a valid OpenAI API key.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

# Select a Product and Embedding Model
This is the product we'll use:

In [2]:
from CONSTANTS import example_asin
import pandas as pd
df_meta = pd.read_parquet('metadata.parquet.gz')
product_name=df_meta.loc[df_meta['asin'] == example_asin]['title'].values[0]

print(product_name)

Streamlight 66118 Stylus Pro LED PenLight with Holster, Black - 100 Lumens


And the embedding model:

In [3]:
from CONSTANTS import embed_model
from langchain.embeddings import OpenAIEmbeddings
langchain_embeddings = OpenAIEmbeddings(model=embed_model)

# Establish Astra Connection

In [4]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {'secure_connect_bundle': os.environ['ASTRA_SECUREBUNDLE_PATH']}
auth_provider = PlainTextAuthProvider(os.environ['ASTRA_CLIENT_ID'], os.environ['ASTRA_CLIENT_SECRET'])
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Create a Vector Store

Langchain provides a `langchain.vectorstores.Cassandra` obect which, while a convenient wrapper around [CassIO](https://cassio.org/), is currently
unable to do metadata filtering. We want to be able to search our store by ASIN (product identifier) and optionally by review rating.

In order to interact with the rest of Langchain in this demo, we simply need a function that is able to return a list of `langchain.docstore.document.Document`
objects, so [`vectorstore.py`](vectorstore.py) provides a `Cassandra` class that does just that.

In [5]:
from CONSTANTS import keyspace_name, table_name
import vectorstore
vstore = vectorstore.Cassandra(langchain_embeddings, session, keyspace_name, table_name)

And from here, we can do text similarity searches:

In [6]:
from CONSTANTS import example_asin
docs=vstore.similarity_search("This flashlight is fantastic", k=100, asin=example_asin, overall_rating=5)
docs[0:10]

[Document(page_content='This flashlight is amazing!', metadata={'asin': 'B0015UC17E', 'overall_rating': 5, 'reviewer_name': 'Ruben Guerrios', 'review_time': '11 June 2015'}),
 Document(page_content='Excellent flashlight', metadata={'asin': 'B0015UC17E', 'overall_rating': 5, 'reviewer_name': 'John E. Salvati', 'review_time': '10 April 2018'}),
 Document(page_content='Great flashlight', metadata={'asin': 'B0015UC17E', 'overall_rating': 5, 'reviewer_name': 'Libby', 'review_time': '25 February 2018'}),
 Document(page_content='Great flashlight', metadata={'asin': 'B0015UC17E', 'overall_rating': 5, 'reviewer_name': 'Marcus M. Zeno Sr.', 'review_time': '04 August 2017'}),
 Document(page_content='Great flashlight', metadata={'asin': 'B0015UC17E', 'overall_rating': 5, 'reviewer_name': 'Robert', 'review_time': '07 November 2014'}),
 Document(page_content='Great flashlight', metadata={'asin': 'B0015UC17E', 'overall_rating': 5, 'reviewer_name': 'Amazon Customer', 'review_time': '08 February 2018'}

# Create a Chat Agent

The Chat Agent will be used to write summaries of reviews and ad copy.


In [7]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from CONSTANTS import chat_model_name

chat = ChatOpenAI(model_name=chat_model_name,temperature=0.2)

# Create and Run a Summary Chain

With our list of 100 documents, we can use `load_summarize_chain` to define a chain that will load the documents, summarize them, and return the
result as a summary.

In [8]:
from langchain.chains.summarize import load_summarize_chain

prompt_template_summary = """
Write a summary of the reviews:

{text}

The summary should be about ten lines long.
"""
PROMPT = PromptTemplate(template=prompt_template_summary, input_variables=["text"])
chain = load_summarize_chain(chat, chain_type="stuff", prompt=PROMPT)
summary=chain.run(docs)

All 100 reviews are thusly summarized:

In [9]:
import pprint
pprint.pprint(summary)

('The reviews for this flashlight are overwhelmingly positive. Many customers '
 'describe it as amazing, excellent, and great. They praise its brightness and '
 'durability, and many mention that it is compact and portable. Some customers '
 'even claim that it is the best flashlight they have ever owned. Overall, the '
 'flashlight receives high praise for its quality, brightness, and '
 'convenience.')


# Social Media Ad

Creating a new prompt, we can run another chain that asks the chat agent to write a social media ad for the product:

In [10]:
prompt_template_fb = """
Write the copy for a single Facebook ad based on the reviews:

{text}

As far as text goes, you can have up to 40 characters in your headline, 
125 characters in your primary text, and 30 characters in your description. 
The primary text should be a bullet-point style, and include emoji.
It description should have a quote from a reviewer.
"""
PROMPT = PromptTemplate(template=prompt_template_fb, input_variables=["text"])
chain = load_summarize_chain(chat, chain_type="stuff", prompt=PROMPT)
fb_copy=chain.run(docs)

print(fb_copy)

🔦 Get the Best Flashlight Ever! 🔦
💡 Bright, Durable, and Compact 💡
⭐️ "Best flashlight I've ever bought." - Happy Customer ⭐️
Don't miss out, order now! 🛒✨


# Send Email

If you have Zapier configured, you can send a marketing email with a call to action. The example here will use a single recipient - our goal
is not to send email to a large number of people, though we could use other Langchain features to do that.

Seach our vector store for a compelling review:

In [11]:
docs=vstore.similarity_search("This flashlight is compact and rugged, I would definitely recommend buying one!", k=1, asin=example_asin, overall_rating=5)
emailBaseDocument = docs[0]
print(emailBaseDocument)

page_content='This little flashlight is amazing! Its compact and is super bright and durable! I will definitely buy another if this one ever gets lost!' metadata={'asin': 'B0015UC17E', 'overall_rating': 5, 'reviewer_name': 'Jesus Ferreira', 'review_time': '03 October 2017'}


Next, build our prompt. We'll reference the 10-line `summary` from before, and the `emailBaseDocument` we just found:

In [ ]:
import os
# You could manually set this here, rather than in the `.env` file.
recipientEmail=os.environ['EMAIL_RECIPIENT']

fromName="Phil"
recipientName="George"

reviewerName=emailBaseDocument.metadata["reviewer_name"]
review=emailBaseDocument.page_content
    
prompt_template_email = f"""
The customer {reviewerName} just gave the following review:

'{review}'

Send a HTML-formatted email to {recipientName} with email address {recipientEmail}, 
based on the review that {reviewerName} gave, and take into account the overall 
summary of the review given here:

'{summary}'

The object of the e-mail is to encourage them to buy the product. Be sure to mention
the customer review.

The email should be signed with {fromName}.
"""

print(prompt_template_email)

We now need to set up an LLM agent, for which we will use the ZapierToolkit:

In [14]:
# Instantiate agent and send emails 
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.agents.agent_toolkits import ZapierToolkit
from langchain.utilities.zapier import ZapierNLAWrapper

zapier = ZapierNLAWrapper()
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)

agent = initialize_agent(toolkit.get_tools(), OpenAI(temperature=0.2), 
        agent="zero-shot-react-description", verbose=True)

And finally, we will have the Agent do it's thing:

In [ ]:
agent.run(prompt_template_email)